In [2]:
import pandas as pd
import glob
import os
from get_densities import *

In [3]:
inp_dir1 = 'img_folder\optic_disc'
inp_dir2 = 'img_folder\macula'
files = glob.glob(inp_dir1 + '/*.png')

In [4]:
mask, ID, gender, exam_ap, examdate, eye, recorddate, exam, scan, prof_den, ves_den = [], [],[],[], [],[],[], [],[],[], []
location = []
names = []
label = []
for file in files:
    image = cv2.imread(file, 0)
    nimage = image.copy()
    nimage = rem_opticdisk(nimage)
    pdy, bin_image = profusion_density(nimage)
    vd, _ = vessel_density(bin_image)
    name = file.split('\\')[-1][:-4].split('_')
    names += [file.split('\\')[-1]]
    mask += [name[0]]
    if int(name[0][-3:])<9:
        label+=['glaucoma']
    else:
        label+=['normal']
    ID += [name[4]]
    gender += [name[5]]
    exam_ap += [name[6]]
    examdate += [name[7]]
    eye += [name[8]]
    recorddate += [name[9]]
    exam += [name[10]]
    scan += [name[11]]
    prof_den += [pdy]
    ves_den += [vd*256/6]
    location += ['Optic Disc']

In [5]:
#mask, ID, gender, exam_ap, examdate, eye, recorddate, exam, scan, prof_den, ves_den = [], [],[],[], [],[],[], [],[],[], []
#location = []
inp_dir2 = 'img_folder\macula'
files = glob.glob(inp_dir2 + '/*.png')
for file in files:
    image = cv2.imread(file, 0)
    pdy, bin_image = profusion_density(image)
    vd, _ = vessel_density(bin_image)
    name = file.split('\\')[-1][:-4].split('_')
    names += [file.split('\\')[-1]]
    mask += [name[0]]
    if int(name[0][-3:])<9:
        label+=['glaucoma']
    else:
        label+=['normal']
    ID += [name[4]]
    gender += [name[5]]
    exam_ap += [name[6]]
    examdate += [name[7]]
    eye += [name[8]]
    recorddate += [name[9]]
    exam += [name[10]]
    scan += [name[11]]
    prof_den += [pdy]
    ves_den += [vd*256/3]
    location += ['Macula']

In [6]:
datadict = {
    'filename' : names,
    'mask' : mask,
    'id' : ID,
    'gender' : gender,
    'exam_type' : exam,
    'edate' : examdate,
    'eye': eye,
    'location' : location,
    'rdate' : recorddate,
    'scan' : scan,
    'label' : label,
    'perfusion_density' : prof_den,
    'vessel_density' : ves_den,
}

In [7]:
df = pd.DataFrame(datadict)

In [8]:
df

,filename,mask,id,gender,exam_type,edate,eye,location,rdate,scan,label,perfusion_density,vessel_density
0,DK0001_DK0001__DK0001_19461120_Male_Angiograph...,DK0001,19461120,Male,Angiography,20211105145649,OD,Optic Disc,20220128135253,Deep,glaucoma,0.461197,11.534505
1,DK0001_DK0001__DK0001_19461120_Male_Angiograph...,DK0001,19461120,Male,Angiography,20211105145649,OD,Optic Disc,20220128135253,Superficial,glaucoma,0.501556,12.052083
2,DK0001_DK0001__DK0001_19461120_Male_Angiograph...,DK0001,19461120,Male,Angiography,20211105145843,OS,Optic Disc,20220128135347,Deep,glaucoma,0.436371,10.565104
3,DK0001_DK0001__DK0001_19461120_Male_Angiograph...,DK0001,19461120,Male,Angiography,20211105145843,OS,Optic Disc,20220128135347,Superficial,glaucoma,0.439331,9.898438
4,DK0002_DK0002__DK0002_19581010_Female_Angiogra...,DK0002,19581010,Female,Angiography,20211118100657,OD,Optic Disc,20220128140717,Deep,glaucoma,0.451920,10.658203
...,...,...,...,...,...,...,...,...,...,...,...,...,...
183,DK0034_DK0034__DK0034_19531104_Female_Angiogra...,DK0034,19531104,Female,Angiography,20190612142237,OS,Macula,20220216141351,Deep,normal,0.468002,18.458333
184,DK0034_DK0034__DK0034_19531104_Female_Angiogra...,DK0034,19531104,Female,Angiography,20211123141815,OS,Macula,20220216141048,Deep,normal,0.491028,19.397135
185,DK0034_DK0034__DK0034_19531104_Female_Angiogra...,DK0034,19531104,Female,Angiography,20211123141815,OS,Macula,20220216141048,Superficial,normal,0.500076,19.692708
186,DK0034_DK0034__DK0034_19531104_Female_Angiogra...,DK0034,19531104,Female,Angiography,20211123141815,OS,Macula,20220216141117,Deep,normal,0.491028,19.397135


In [9]:
df['dob'] = float('nan')
df['exam_date'] = float('nan')
df['exam_time'] = float('nan')
df['record_date'] = float('nan')
df['record_time'] = float('nan')

df['dob'] = df.apply(lambda x : pd.to_datetime(x['id'], format="%Y%m%d").date(), axis = 1)
df['exam_date'] = df.apply(lambda x : pd.to_datetime(x['edate'], format="%Y%m%d%H%M%S").date(), axis = 1)
df['exam_time'] = df.apply(lambda x : pd.to_datetime(x['edate'], format="%Y%m%d%H%M%S").time(), axis = 1)
df['record_date'] = df.apply(lambda x : pd.to_datetime(x['rdate'], format="%Y%m%d%H%M%S").date(), axis = 1)
df['record_time'] = df.apply(lambda x : pd.to_datetime(x['rdate'], format="%Y%m%d%H%M%S").time(), axis = 1)

In [10]:
examdf = df[['filename', 'mask', 'dob', 'gender', 'exam_type', 'exam_date', 'exam_time', 'eye', 'location', 'record_date', 'record_time', 'scan', 'label','perfusion_density', 'vessel_density']]

In [11]:
ndf = examdf[examdf['exam_type']!='Structure']

In [12]:
ndf.to_csv('octa_new.csv',index=False)

In [12]:
pd.to_datetime(examdate[0], format="%Y%m%d%H%M%S").time()

datetime.time(14, 56, 49)

In [12]:
ndf

,filename,mask,dob,gender,exam_type,exam_date,exam_time,eye,location,record_date,record_time,scan,perfusion_density,vessel_density
0,DK0001_DK0001__DK0001_19461120_Male_Angiograph...,DK0001,1946-11-20,Male,Angiography,2021-11-05,14:56:49,OD,Optic Disc,2022-01-28,13:52:53,Deep,0.389114,9.686849
1,DK0001_DK0001__DK0001_19461120_Male_Angiograph...,DK0001,1946-11-20,Male,Angiography,2021-11-05,14:56:49,OD,Optic Disc,2022-01-28,13:52:53,Superficial,0.435394,10.399089
2,DK0001_DK0001__DK0001_19461120_Male_Angiograph...,DK0001,1946-11-20,Male,Angiography,2021-11-05,14:58:43,OS,Optic Disc,2022-01-28,13:53:47,Deep,0.355606,8.462240
3,DK0001_DK0001__DK0001_19461120_Male_Angiograph...,DK0001,1946-11-20,Male,Angiography,2021-11-05,14:58:43,OS,Optic Disc,2022-01-28,13:53:47,Superficial,0.366577,8.187500
4,DK0002_DK0002__DK0002_19581010_Female_Angiogra...,DK0002,1958-10-10,Female,Angiography,2021-11-18,10:06:57,OD,Optic Disc,2022-01-28,14:07:17,Deep,0.382919,8.973307
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
98,DK0011_DK0011__DK0011_19490211_Female_Angiogra...,DK0011,1949-02-11,Female,Angiography,2019-09-26,13:41:22,OS,Macula,2022-02-14,14:07:10,Deep,0.381836,14.865885
99,DK0011_DK0011__DK0011_19490211_Female_Angiogra...,DK0011,1949-02-11,Female,Angiography,2020-02-20,11:40:16,OD,Macula,2022-02-14,14:01:29,Deep,0.365616,14.928385
100,DK0011_DK0011__DK0011_19490211_Female_Angiogra...,DK0011,1949-02-11,Female,Angiography,2020-02-20,11:40:16,OD,Macula,2022-02-14,14:01:29,Superficial,0.397980,15.430990
101,DK0011_DK0011__DK0011_19490211_Female_Angiogra...,DK0011,1949-02-11,Female,Angiography,2020-02-20,11:43:01,OS,Macula,2022-02-14,14:04:02,Deep,0.378052,15.263021


In [21]:
0.460083

30151.999488

In [23]:
0.255280*256/6

10.891946666666668

In [ ]:
 PIL